In [79]:
import pandas as pd
import numpy as np
import sklearn

In [80]:
data_food = pd.read_csv("DataFood.csv")
data_country = pd.read_csv("DataCountry.csv")

In [81]:
data_food.shape

(25416, 18)

In [82]:
data_country.shape

(214, 12)

In [83]:
# Before Processing
data_food.head()

,m49_code,country,region,cpc_code,commodity,year,loss_percentage,loss_percentage_original,loss_quantity,activity,food_supply_stage,treatment,cause_of_loss,sample_size,method_data_collection,reference,url,notes
0,104,Myanmar,NaN,23161.02,"Rice, milled",2015,1.78,1.78%,26.12kgs,Storage,Storage,"30 days storage, with trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
1,104,Myanmar,NaN,23161.02,"Rice, milled",2015,11.77,11.77%,88.18kgs,Storage,Storage,"60 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
2,104,Myanmar,NaN,23161.02,"Rice, milled",2015,5.88,5.88%,44.09kgs,Storage,Storage,"30 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
3,104,Myanmar,NaN,23161.02,"Rice, milled",2015,3.57,3.57%,52.24kgs,Storage,Storage,"60 days storage, with trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
4,104,Myanmar,NaN,23161.02,"Rice, milled",2015,17.65,17.65%,132.27kgs,Storage,Storage,"90 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically


In [84]:
data_food = data_food.drop(columns=['reference', 'url', 'notes','country'])

In [85]:
data_food.dtypes

m49_code                      int64
region                       object
cpc_code                     object
commodity                    object
year                          int64
loss_percentage             float64
loss_percentage_original     object
loss_quantity                object
activity                     object
food_supply_stage            object
treatment                    object
cause_of_loss                object
sample_size                  object
method_data_collection       object
dtype: object

In [86]:
data_food.shape

(25416, 14)

In [87]:
# Before Processing
data_country.head()

,Country,combined figures (kg/capita/year),Household estimate (kg/capita/year),Household estimate (tonnes/year),Retail estimate (kg/capita/year),Retail estimate (tonnes/year),Food service estimate (kg/capita/year),Food service estimate (tonnes/year),Confidence in estimate,M49 code,Region,Source
0,Afghanistan,126,82,3109153,16,594982,28,1051783,Very Low Confidence,4,Southern Asia,https://www.unep.org/resources/report/unep-foo...
1,Albania,127,83,238492,16,45058,28,79651,Very Low Confidence,8,Southern Europe,https://www.unep.org/resources/report/unep-foo...
2,Algeria,135,91,3918529,16,673360,28,1190335,Very Low Confidence,12,Northern Africa,https://www.unep.org/resources/report/unep-foo...
3,Andorra,123,84,6497,13,988,26,1971,Low Confidence,20,Southern Europe,https://www.unep.org/resources/report/unep-foo...
4,Angola,144,100,3169523,16,497755,28,879908,Very Low Confidence,24,Sub-Saharan Africa,https://www.unep.org/resources/report/unep-foo...


In [88]:
data_country = data_country.drop(columns=['Source', 'Household estimate (kg/capita/year)','Retail estimate (kg/capita/year)','Food service estimate (kg/capita/year)'])
data_country['combined figures (tonnes/year)'] = data_country['Household estimate (tonnes/year)'] + data_country['Retail estimate (tonnes/year)'] + data_country['Food service estimate (tonnes/year)'] 
data_country = data_country.drop(columns=['Household estimate (tonnes/year)', 'Retail estimate (tonnes/year)', 'Food service estimate (tonnes/year)'])
data_country = data_country.rename(columns={"M49 code":"m49_code", 
                                            "combined figures (kg/capita/year)":"total_kg/capita/year", 
                                            "Confidence in estimate": "conf_in_estimate"

                                            })


In [89]:
data_country.head()

,Country,total_kg/capita/year,conf_in_estimate,m49_code,Region,combined figures (tonnes/year)
0,Afghanistan,126,Very Low Confidence,4,Southern Asia,4755918
1,Albania,127,Very Low Confidence,8,Southern Europe,363201
2,Algeria,135,Very Low Confidence,12,Northern Africa,5782224
3,Andorra,123,Low Confidence,20,Southern Europe,9456
4,Angola,144,Very Low Confidence,24,Sub-Saharan Africa,4547186


In [90]:
data_country.dtypes

Country                           object
total_kg/capita/year               int64
conf_in_estimate                  object
m49_code                           int64
Region                            object
combined figures (tonnes/year)     int64
dtype: object

In [91]:
# Merged data
data_food_country = pd.merge(data_food,data_country, on='m49_code')
data_food_country = data_food_country.drop(columns=['m49_code','sample_size','region','cause_of_loss','treatment','activity','method_data_collection','total_kg/capita/year','loss_quantity'])
data_food_country = data_food_country.dropna(subset=['food_supply_stage'])

In [92]:
data_food_country.head()

,cpc_code,commodity,year,loss_percentage,loss_percentage_original,food_supply_stage,Country,conf_in_estimate,Region,combined figures (tonnes/year)
0,23161.02,"Rice, milled",2015,1.78,1.78%,Storage,Myanmar,Very Low Confidence,South-eastern Asia,7005658
1,23161.02,"Rice, milled",2015,11.77,11.77%,Storage,Myanmar,Very Low Confidence,South-eastern Asia,7005658
2,23161.02,"Rice, milled",2015,5.88,5.88%,Storage,Myanmar,Very Low Confidence,South-eastern Asia,7005658
3,23161.02,"Rice, milled",2015,3.57,3.57%,Storage,Myanmar,Very Low Confidence,South-eastern Asia,7005658
4,23161.02,"Rice, milled",2015,17.65,17.65%,Storage,Myanmar,Very Low Confidence,South-eastern Asia,7005658


In [93]:
data_food_country = data_food_country.rename(columns={"Region":"region"})


In [94]:
# Some data were lost because rows do not match
data_food_country.shape

(21956, 10)

In [95]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector


In [96]:
target_name = 'food_supply_stage'
target = data_food_country[target_name]
data = data_food_country.drop(columns=[target_name])




numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)



categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)


model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

_ = model.fit(data_train, target_train)
data_test.head()

,cpc_code,commodity,year,loss_percentage,loss_percentage_original,Country,conf_in_estimate,region,combined figures (tonnes/year)
19247,0112,Maize (corn),2011,6.37256,6.37256,Botswana,Very Low Confidence,Sub-Saharan Africa,311525
23652,01213,Cauliflowers and broccoli,2002,8.00000,8,United States of America,High Confidence,Northern America,45446091
7605,01460,"Coconuts, in shell",2014,0.15000,0.15,India,Very Low Confidence,Southern Asia,127910071
10070,0112,Maize (corn),2021,2.13245,2.13245,Lesotho,Very Low Confidence,Sub-Saharan Africa,303661
170,0111,Wheat,2014,4.86667,4.86667,Burundi,Very Low Confidence,Sub-Saharan Africa,1683267


In [101]:
model.predict(data_test)

array(['Farm', 'Whole supply chain', 'Storage', ..., 'Transport', 'Farm',
       'Farm'], dtype=object)

In [102]:
model.score(data_test, target_test)

0.8261978502459464